In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip "/content/drive/My Drive/Content-Summary_CleanData.zip" -d "/content/extracted_files"

Archive:  /content/drive/My Drive/Content-Summary_CleanData.zip
  inflating: /content/extracted_files/Content-Summary_CleanData.csv  


In [ ]:
import pandas as pd

df = pd.read_csv('/content/extracted_files/Content-Summary_CleanData.csv')
df.head()

,Content,Summary
0,New York police are concerned drones could bec...,Police have investigated criminals who have ri...
1,By . Ryan Lipman . Perhaps Australian porn sta...,Porn star Angela White secretly filmed sex act...
2,"This was, Sergio Garcia conceded, much like be...",American draws inspiration from fellow country...
3,An Ebola outbreak that began in Guinea four mo...,World Health Organisation: 635 infections and ...
4,By . Associated Press and Daily Mail Reporter ...,A sinkhole opened up at 5:15am this morning in...


In [ ]:
#  install unsloth , xformers (flash attention) and all other packages !
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# we have to check which torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"

!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-uvid1g_9/unsloth_797177298acc413bb05666cc732b06ee
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-uvid1g_9/unsloth_797177298acc413bb05666cc732b06ee
  Resolved https://github.com/unslothai/unsloth.git to commit bdf0cd6033595be4e7ed23d0d002bb176d343152
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 6.0 MB/s eta 0:00:00
   

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 46.3 MB/s eta 0:00:00


In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model,tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.7: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2025.1.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Data Prep

In [ ]:
# reducing data set from 600k to 10000
df_subset = df.iloc[:10000]
df_subset.to_csv("summary10000.csv", index=False)

In [ ]:
df_10000 = pd.read_csv('/content/summary10000.csv')

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset
# Split your DataFrame into training and testing sets (80% train, 20% test)
train_df, test_df = train_test_split(df_10000, test_size=0.2)

# Save the splits to separate CSV files
train_df.to_csv('train_dataset.csv', index=False)
test_df.to_csv('test_dataset.csv', index=False)

# Load the datasets
train_dataset = load_dataset("csv", data_files="train_dataset.csv")
test_dataset = load_dataset("csv", data_files="test_dataset.csv")

print(train_dataset['train'])
print(test_dataset['train'])

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['Content', 'Summary'],
    num_rows: 8000
})
Dataset({
    features: ['Content', 'Summary'],
    num_rows: 2000
})


In [ ]:
train_dataset['train']

Dataset({
    features: ['Content', 'Summary'],
    num_rows: 8000
})

In [ ]:
#convert the dataset to dataframe
train_df = train_dataset['train'].to_pandas(train_dataset['train'])

In [ ]:
train_df.shape

(8000, 2)

In [ ]:
import pandas as pd
from datasets import Dataset

# convert the dataframe to dataset
dataset = Dataset.from_pandas(train_df)

In [ ]:
dataset

Dataset({
    features: ['Content', 'Summary'],
    num_rows: 8000
})

In [ ]:
test_dataset

DatasetDict({
    train: Dataset({
        features: ['Content', 'Summary'],
        num_rows: 2000
    })
})

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Your task is to understand the provided text in English and generate a summary for it in english. The result should be formatted in JSON. Follow these steps:

1- Comprehension:
Understand the provided English text.
2- Summary Generation:
Create a concise summary in english under 60 words.
3-Output Formatting:
Format the summary into a JSON object with the key 'summary'.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs       = examples["Content"]
    outputs      = examples["Summary"]
    texts = []
    for input, output in zip(inputs, outputs):
        # must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass



In [ ]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

## Training Model

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,         # reducing steps
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/8000 [00:00<?, ? examples/s]

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")


GPU = Tesla T4. Max memory = 14.748 GB.
5.848 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 8,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
1,2.126200
2,2.292600
3,2.223000
4,2.165800
5,1.996300
6,2.073300
7,1.978900
8,1.869800
9,1.900400
10,1.797200


In [ ]:
trainer.save_model('/content/news_summary_model_atul_latest')

In [ ]:
# article  = df.sample(1).iloc[0]['Content']

article ="""
The decision to buy a flagship or luxury product is often driven more by brand value than the product’s true value, a phenomenon known as brand bias. As a result, we may choose a less superior option even when better alternatives are available at a better value. In the smartphone industry, iQOO stands out as a brand that focuses on delivering exceptional value that some might overlook. Over the years, iQOO has been consistently releasing flagship Android smartphones that outperform the competition while being much more affordable, and their latest model, the iQOO 13, continues this trend. It offers incredible value without compromising on the key features essential in a flagship device today. You have exhausted your monthly limit of free stories. Register to read more stories Sign up Already have an account? Sign in
"""

In [ ]:

# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        article, # input
        "", # output - leaving this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYour task is to understand the provided text in English and generate a summary for it in english. The result should be formatted in JSON. Follow these steps:\n\n1- Comprehension:\nUnderstand the provided English text.\n2- Summary Generation:\nCreate a concise summary in english under 60 words.\n3-Output Formatting:\nFormat the summary into a JSON object with the key'summary'.\n\n### Input:\n\nThe decision to buy a flagship or luxury product is often driven more by brand value than the product’s true value, a phenomenon known as brand bias. As a result, we may choose a less superior option even when better alternatives are available at a better value. In the smartphone industry, iQOO stands out as a brand that focuses on delivering exceptional value that some might overlook. Over the years, i

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        article, # input
        "", # output - leaving this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 400)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Your task is to understand the provided text in English and generate a summary for it in english. The result should be formatted in JSON. Follow these steps:

1- Comprehension:
Understand the provided English text.
2- Summary Generation:
Create a concise summary in english under 60 words.
3-Output Formatting:
Format the summary into a JSON object with the key'summary'.

### Input:

The decision to buy a flagship or luxury product is often driven more by brand value than the product’s true value, a phenomenon known as brand bias. As a result, we may choose a less superior option even when better alternatives are available at a better value. In the smartphone industry, iQOO stands out as a brand that focuses on delivering exceptional value that some might overlook. Over the years, iQOO has been con

In [ ]:

# Enable native 2x faster inference
FastLanguageModel.for_inference(model)

# Prepare inputs for the model
inputs = tokenizer(
    [
        alpaca_prompt.format(
            article,  # input
            "",       # output - leaving this blank for generation!
        )
    ], return_tensors="pt"
).to("cuda")

# Generate outputs
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

# Decode the generated outputs
decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Store the output in a variable
summary_output = decoded_output[0]  # Assuming you want the first output

# Now, you can use summary_output as needed
print(summary_output)  # This will display the generated summary


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Your task is to understand the provided text in English and generate a summary for it in english. The result should be formatted in JSON. Follow these steps:

1- Comprehension:
Understand the provided English text.
2- Summary Generation:
Create a concise summary in english under 60 words.
3-Output Formatting:
Format the summary into a JSON object with the key'summary'.

### Input:

The decision to buy a flagship or luxury product is often driven more by brand value than the product’s true value, a phenomenon known as brand bias. As a result, we may choose a less superior option even when better alternatives are available at a better value. In the smartphone industry, iQOO stands out as a brand that focuses on delivering exceptional value that some might overlook. Over the years, iQOO has been consistently releasi

In [ ]:
import re

match = re.search(r'### Response:\s*(.*)', summary_output)

if match:
    response_line = match.group(1).strip()  # Extract and strip leading/trailing whitespace
    print(response_line)
else:
    print("No response found.")

iQOO 13 is a flagship smartphone that offers exceptional value and top-of-the-line features, including a powerful processor, a high refresh rate display, and a versatile camera system.


In [ ]:
# !pip install evaluate
# !pip install rouge-score


In [ ]:
print(dataset[0:1]['Content'])

print("\n\n")


print(dataset[0:1]['Summary'])


['Regina Hungerford, 55, denied striking the 17-year-old when she lost her temper as he listened to music in her classroom at Merthyr Tydfil College.\nShe previously had a common assault conviction quashed on appeal, saying the notebook was slammed on the desk.\nBut on Friday, the Education Workforce Council ruled contact with the pupil was probably made and was unacceptable.\nIt will decide whether to strike her off the register at a later date.\nPanel chairman Gareth Roberts said: "There was a momentary lapse of control and in that moment Mrs Hungerford acted in breach of teaching principles.\n"It put at risk the public trust in the profession and her conduct fell far short of the standard expected of a registered teacher.\n"Pupil A\'s behaviour was difficult to manage but we have concluded she acted in an inappropriate way."\nDuring the hearing, Mrs Hungerford\'s lawyer Adam Ohringer said her "calm and patient" experience with challenging students meant she would not have struck the

In [ ]:
test_article = dataset[0:1]['Content']



FastLanguageModel.for_inference(model)

# Prepare inputs for the model
inputs = tokenizer(
    [
        alpaca_prompt.format(
            test_article,  # input
            "",       # output - leaving this blank for generation!
        )
    ], return_tensors="pt"
).to("cuda")

# Generate outputs
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

# Decode the generated outputs
decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Store the output in a variable
summary_output = decoded_output[0]  # Assuming you want the first output

# Now, you can use summary_output as needed
print(summary_output)  # This will display the generated summary


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Your task is to understand the provided text in English and generate a summary for it in english. The result should be formatted in JSON. Follow these steps:

1- Comprehension:
Understand the provided English text.
2- Summary Generation:
Create a concise summary in english under 60 words.
3-Output Formatting:
Format the summary into a JSON object with the key'summary'.

### Input:
['Regina Hungerford, 55, denied striking the 17-year-old when she lost her temper as he listened to music in her classroom at Merthyr Tydfil College.\nShe previously had a common assault conviction quashed on appeal, saying the notebook was slammed on the desk.\nBut on Friday, the Education Workforce Council ruled contact with the pupil was probably made and was unacceptable.\nIt will decide whether to strike her off the register at a l

In [ ]:
import re

match = re.search(r'### Response:\s*(.*)', summary_output)

if match:
    response_line = match.group(1).strip()  # Extract and strip leading/trailing whitespace
    print(response_line)
else:
    print("No response found.")

A teacher who hit a pupil with a notebook in a Merthyr Tydfil classroom has been found guilty of unacceptable professional conduct.


In [ ]:
from transformers import TextStreamer

class CustomTextStreamer(TextStreamer):
    def __init__(self, tokenizer):
        super().__init__(tokenizer)
        self.generated_text = ""

    def on_text(self, text):
        self.generated_text += text



# enable native 2x faster inference
FastLanguageModel.for_inference(model)

# Prepare inputs for the model
inputs = tokenizer(
    [
        alpaca_prompt.format(
            article,  # input
            "",       # output - leaving this blank for generation
        )
    ], return_tensors="pt"
).to("cuda")


custom_streamer = CustomTextStreamer(tokenizer)

# generate outputs with the custom streamer
_ = model.generate(**inputs, streamer=custom_streamer, max_new_tokens=400)

# Now we can access the generated text from the custom streamer
summary_output = custom_streamer.generated_text

print(summary_output)


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Your task is to understand the provided text in English and generate a summary for it in english. The result should be formatted in JSON. Follow these steps:

1- Comprehension:
Understand the provided English text.
2- Summary Generation:
Create a concise summary in english under 60 words.
3-Output Formatting:
Format the summary into a JSON object with the key'summary'.

### Input:

The decision to buy a flagship or luxury product is often driven more by brand value than the product’s true value, a phenomenon known as brand bias. As a result, we may choose a less superior option even when better alternatives are available at a better value. In the smartphone industry, iQOO stands out as a brand that focuses on delivering exceptional value that some might overlook. Over the years, iQOO has been con